In [59]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


Carga de los datasets

In [60]:
movies_df = pd.read_csv(r"E:\Repositorios y bases de datos\Henry DS\Proyecto-individual-N-1---LABS---Henry\datos_procesados\movies_df.csv")
director_actor_df = pd.read_csv(r"E:\Repositorios y bases de datos\Henry DS\Proyecto-individual-N-1---LABS---Henry\datos_procesados\director_actor_df.csv")

Reducción del dataset específicamente para el modelo de recomendación

In [61]:
movies_df_modelo = movies_df.dropna() 
movies_df_modelo = movies_df_modelo[movies_df_modelo['return'] != 0]
movies_df_modelo = movies_df_modelo[movies_df_modelo['original_title'].apply(lambda x: x.isascii() if isinstance(x, str) else False)]

In [62]:
# Eliminación de columnas que pueden haber sido útiles para los endpoints pero que no usaría como parámentro para recomendar
movies_df_modelo = movies_df_modelo.drop(columns=['original_title', 'release_date', 'revenue', 'vote_count', 'return'])

# Muestra del DataFrame resultante
movies_df_modelo

,budget,movie_id,title,vote_average,genre
0,30000000.0,862,Toy Story,7.7,Animation
1,65000000.0,8844,Jumanji,6.9,Adventure
3,16000000.0,31357,Waiting to Exhale,6.1,Comedy
5,60000000.0,949,Heat,7.7,Action
8,35000000.0,9091,Sudden Death,5.5,Action
...,...,...,...,...,...
44970,2196531.0,26147,FC Venus,5.6,Comedy
45014,60000000.0,353491,The Dark Tower,5.7,Action
45139,50000000.0,378236,The Emoji Movie,5.8,Comedy
45167,11000000.0,395834,Wind River,7.4,Action


In [63]:
# Conversión de 'movie_id' a tipo numérico en ambos DataFrames
director_actor_df['movie_id'] = pd.to_numeric(director_actor_df['movie_id'], errors='coerce')
movies_df_modelo['movie_id'] = pd.to_numeric(movies_df_modelo['movie_id'])

# Unión utilizando los 'movie_id' comunes
director_actor_df_modelo = pd.merge(director_actor_df, movies_df_modelo[['movie_id']], on='movie_id', how='inner')
director_actor_df_modelo = director_actor_df_modelo.drop("Unnamed: 0", axis=1)


director_actor_df_modelo

,movie_id,director,actor
0,862,John Lasseter,"['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim..."
1,8844,Joe Johnston,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D..."
2,31357,Forest Whitaker,"['Whitney Houston', 'Angela Bassett', 'Loretta..."
3,949,Michael Mann,"['Al Pacino', 'Robert De Niro', 'Val Kilmer', ..."
4,9091,Peter Hyams,"['Jean-Claude Van Damme', 'Powers Boothe', 'Do..."
...,...,...,...
5075,26147,Joona Tena,"['Minna Haapkylä', 'Petteri Summanen', 'Laura ..."
5076,353491,Nikolaj Arcel,"['Idris Elba', 'Matthew McConaughey', 'Tom Tay..."
5077,378236,Anthony Leondis,"['T.J. Miller', 'James Corden', 'Anna Faris', ..."
5078,395834,Taylor Sheridan,"['Jeremy Renner', 'Elizabeth Olsen', 'Gil Birm..."


In [64]:
movies_df_modelo

,budget,movie_id,title,vote_average,genre
0,30000000.0,862,Toy Story,7.7,Animation
1,65000000.0,8844,Jumanji,6.9,Adventure
3,16000000.0,31357,Waiting to Exhale,6.1,Comedy
5,60000000.0,949,Heat,7.7,Action
8,35000000.0,9091,Sudden Death,5.5,Action
...,...,...,...,...,...
44970,2196531.0,26147,FC Venus,5.6,Comedy
45014,60000000.0,353491,The Dark Tower,5.7,Action
45139,50000000.0,378236,The Emoji Movie,5.8,Comedy
45167,11000000.0,395834,Wind River,7.4,Action


In [65]:
# Fusión de df_movies y director_actor_df en base a movie_id
movies_data = pd.merge(movies_df_modelo, director_actor_df_modelo, on='movie_id', how='left')

# Verificación de las columnas disponibles en movies_data
print(movies_data.columns)


Index(['budget', 'movie_id', 'title', 'vote_average', 'genre', 'director',
       'actor'],
      dtype='object')


Vectorización y similitud del coseno

In [66]:
reference_movie = 'Toy Story'

# Vectorización TF-IDF de las características relevantes (género, director y actor)
vectorizer = TfidfVectorizer(stop_words='english')
text_data = movies_data['genre'] + ' ' + movies_data['director'].fillna('') + ' ' + movies_data['actor'].fillna('')
tfidf_matrix = vectorizer.fit_transform(text_data)

# Cálculo de la similitud de coseno entre la película de referencia y todas las películas
reference_index = movies_data[movies_data['title'] == reference_movie].index[0]
similarities = cosine_similarity(tfidf_matrix[reference_index], tfidf_matrix).flatten()

# Películas más similares ordenadas (excluyendo la película de referencia)
similar_movies_indices = similarities.argsort()[::-1][1:6]  # Excluye la primera (la misma película)
recommended_movies = movies_data.iloc[similar_movies_indices]['title'].tolist()

# Print de las películas recomendadas
print(f"Películas recomendadas para '{reference_movie}':")
for movie in recommended_movies:
    print(movie)


Películas recomendadas para 'Toy Story':
Toy Story 2
Toy Story 3
Atlantis: The Lost Empire
Cars
Cars 2
